# Collision Avoidance - Train Model

Welcome to this host side Jupyter Notebook!  This should look familiar if you ran through the notebooks that run on the robot.  In this notebook we'll train our image classifier to detect two classes
``free`` and ``blocked``, which we'll use for avoiding collisions.  For this, we'll use a popular deep learning library *PyTorch*

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

### Upload and extract dataset

Before you start, you should upload the ``dataset.zip`` file that you created in the ``data_collection.ipynb`` notebook on the robot.

You should then extract this dataset by calling the command below

In [3]:
#!unzip -q dataset.zip

You should see a folder named ``dataset`` appear in the file browser.

### Create dataset instance

Now we use the ``ImageFolder`` dataset class available with the ``torchvision.datasets`` package.  We attach transforms from the ``torchvision.transforms`` package to prepare the data for training.  

In [4]:
dataset = datasets.ImageFolder(
    'dataset',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

print(len(dataset))
print(dataset.classes)
      
# for elem in dataset:
#    print(elem)

49
['left', 'right', 'stop', 'straight']


### Split dataset into train and test sets

Next, we split the dataset into *training* and *test* sets.  The test set will be used to verify the accuracy of the model we train.

In [6]:
train_len=int(round(len(dataset)*0.4, 0))
test_len=len(dataset)-train_len
print(train_len,test_len)

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_len, test_len])

39 10


### Create data loaders to load data in batches

We'll create two ``DataLoader`` instances, which provide utilities for shuffling data, producing *batches* of images, and loading the samples in parallel with multiple workers.

In [7]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

print(len(test_dataset))
print(len(train_dataset))



10
39


### Define the neural network

Now, we define the neural network we'll be training.  The *torchvision* package provides a collection of pre-trained models that we can use.

In a process called *transfer learning*, we can repurpose a pre-trained model (trained on millions of images) for a new task that has possibly much less data available.

Important features that were learned in the original training of the pre-trained model are re-usable for the new task.  We'll use the ``alexnet`` model.

In [8]:
model = models.alexnet(pretrained=True)


The ``alexnet`` model was originally trained for a dataset that had 1000 class labels, but our dataset only has two class labels!  We'll replace
the final layer with a new, untrained layer that has only two outputs.  

In [9]:
#print(model)

In [10]:
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 4)

Finally, we transfer our model for execution on the GPU

In [11]:
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p

In [12]:
device = torch.device('cuda')
model = model.to(device)

print(torch.version.cuda)
print(device)

print(len(train_loader.dataset))

10.0.117
cuda
39


### Train the neural network

Using the code below we will train the neural network for 30 epochs, saving the best performing model after each epoch.

> An epoch is a full run through our data.

In [13]:
NUM_EPOCHS = 30
BEST_MODEL_PATH = 'best_model.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        print('Train', len(labels), len(images))
        optimizer.zero_grad()
        outputs = model(images)
#        print(outputs)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        print('Test')
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy

Train 16 16
Train 16 16
Train 7 7
Test
0: 0.300000
Train 16 16
Train 16 16
Train 7 7
Test
1: 0.600000
Train 16 16
Train 16 16
Train 7 7
Test
2: 0.900000
Train 16 16
Train 16 16
Train 7 7
Test
3: 0.400000
Train 16 16
Train 16 16
Train 7 7
Test
4: 0.800000
Train 16 16
Train 16 16
Train 7 7
Test
5: 0.700000
Train 16 16
Train 16 16
Train 7 7
Test
6: 0.800000
Train 16 16
Train 16 16
Train 7 7
Test
7: 0.700000
Train 16 16
Train 16 16
Train 7 7
Test
8: 0.700000
Train 16 16
Train 16 16
Train 7 7
Test
9: 0.700000
Train 16 16
Train 16 16
Train 7 7
Test
10: 0.600000
Train 16 16
Train 16 16
Train 7 7
Test
11: 0.900000
Train 16 16
Train 16 16
Train 7 7
Test
12: 0.900000
Train 16 16
Train 16 16
Train 7 7
Test
13: 1.000000
Train 16 16
Train 16 16
Train 7 7
Test
14: 0.700000
Train 16 16
Train 16 16
Train 7 7
Test
15: 0.900000
Train 16 16
Train 16 16
Train 7 7
Test
16: 0.600000
Train 16 16
Train 16 16
Train 7 7
Test
17: 0.800000
Train 16 16
Train 16 16
Train 7 7
Test
18: 0.600000
Train 16 16
Train 16 1

Once that is finished, you should see a file ``best_model.pth`` in the Jupyter Lab file browser.  Select ``Right click`` -> ``Download`` to download the model to your workstation